In [2]:
import pandas as pd
import numpy as np
from scripts.pipeline_steps import step5 as s5
from scripts.notebook_operations import setup_notebook
from scripts.globals import MUTSIG_PROBABILITIES

engine, g37, transcripts, genes, mirnas = setup_notebook()



import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
sns.set_palette("deep")

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


In [3]:
df = s5.apply_step_5('results/test_simulated_1/results_test_simulated.csv', g37, MUTSIG_PROBABILITIES)

In [4]:
len(df)

1742

In [ ]:
df

In [3]:

mut_threshold, wt_threshold = (0.5 , 0.5)
df = s5.filter_rows_by_thresholds(df, mut_threshold, wt_threshold)

# add cols
df["is_gain"] = df.mut_prediction > df.wt_prediction
df["is_gene_upregulated"] = ~df.is_gain



# merge 
df = df.merge(genes, on="gene_id", how="left")
df = df.merge(mirnas, on="mirna_accession", how="left")
df.drop(columns=["chr", "start", "end", "sequence", "conservation", "mirna_name_cmc"], inplace=True)

# fill NAs
df["mutsig"] = df["mutsig"].fillna("not_found")
df["gene_id"] = df["gene_id"].fillna("not_found")
df["gene_name"] = df["gene_name"].fillna("not_found")
df["gene_biotype"] = df["gene_biotype"].fillna("not_found")
df["gene_description"] = df["gene_description"].fillna("not_found")
df = df.fillna(False)

df.head()

/tmp/ipykernel_12594/278997352.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


,id,wt_prediction,mut_prediction,pred_difference,vcf_id,mirna_accession,gene_id,mutation_context,mutsig,is_intron,is_gain,is_gene_upregulated,gene_name,gene_biotype,gene_description,is_driver_intogen,tier_cosmic,is_hallmark_cosmic,is_fusion,is_oncogene,is_tsupp,is_gene_of_interest,mirna_name,mirna_family,seed,is_mirna_oncogene,is_mirna_tumor_suppressor
0,PD3851a+_3_1992673_T_C_MIMAT0000075,0.429814,0.746308,0.316,PD3851a+,MIMAT0000075,not_found,G[T>C]C,not_found,False,True,False,not_found,not_found,not_found,False,False,False,False,False,False,False,hsa-miR-20a-5p,miR-17-5p/20-5p/93-5p/106-5p/519-3p,TACCTGC,True,False
1,PD3851a+_3_1992673_T_C_MIMAT0000093,0.412005,0.666354,0.254,PD3851a+,MIMAT0000093,not_found,G[T>C]C,not_found,False,True,False,not_found,not_found,not_found,False,False,False,False,False,False,False,hsa-miR-93-5p,miR-17-5p/20-5p/93-5p/106-5p/519-3p,TACCTGC,True,False
2,PD3851a+_3_1992673_T_C_MIMAT0000098,0.681526,0.446319,-0.235,PD3851a+,MIMAT0000098,not_found,G[T>C]C,not_found,False,False,True,not_found,not_found,not_found,False,False,False,False,False,False,False,hsa-miR-100-5p,miR-99-5p/100-5p,ACAAGTT,False,True
3,PD3851a+_3_1992673_T_C_MIMAT0000680,0.743227,0.279636,-0.464,PD3851a+,MIMAT0000680,not_found,G[T>C]C,not_found,False,False,True,not_found,not_found,not_found,False,False,False,False,False,False,False,hsa-miR-106b-5p,miR-17-5p/20-5p/93-5p/106-5p/519-3p,TCTGCAC,True,False
4,PD3851a+_3_1992673_T_C_MIMAT0001413,0.381469,0.706472,0.325,PD3851a+,MIMAT0001413,not_found,G[T>C]C,not_found,False,True,False,not_found,not_found,not_found,False,False,False,False,False,False,False,hsa-miR-20b-5p,miR-17-5p/20-5p/93-5p/106-5p/519-3p,TACCTGC,False,False
